In [22]:
!pip install python-dotenv
!pip install requests
import os
import requests
from dotenv import load_dotenv
load_dotenv()  # Carrega o .env do diretório atual


True

In [23]:
import zipfile
import os

# Caminhos
zip_file_path = "/content/202401_NFs.zip"  # Arquivo ZIP
extraction_path = "/content/dados_extraidos/"  # Pasta de destino

# Criar pasta se não existir
os.makedirs(extraction_path, exist_ok=True)

# Extrair
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)
    print(f"Arquivos extraídos em: {extraction_path}")

# Verificar conteúdo
!ls -lh {extraction_path}

Arquivos extraídos em: /content/dados_extraidos/
total 336K
-rw-r--r-- 1 root root  43K Jun 16 19:13 202401_NFs_Cabecalho.csv
-rw-r--r-- 1 root root 292K Jun 16 19:13 202401_NFs_Itens.csv


In [24]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()  # Carrega o arquivo .env
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")


In [25]:
import sys
!{sys.executable} -m pip install langchain-experimental
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOpenAI
import glob
import pandas as pd

diretorio = glob.glob("dados_extraidos/*.csv")
dfs_csv = []

for arquivo in diretorio:
    df = pd.read_csv(arquivo)
    df['Arquivo_Origem'] = arquivo.split('/')[-1]  # Adiciona o nome do arquivo como coluna
    dfs_csv.append(df)

df_final = pd.concat(dfs_csv, ignore_index=True)
print(df_final.head())

# 1. Configuração segura do agente
agent_executor = create_pandas_dataframe_agent(
    ChatOpenAI(
        openai_api_key=deepseek_api_key,
        openai_api_base="https://api.deepseek.com/v1",
        model_name="deepseek-chat",
        temperature=0.3,
        max_tokens=2000
    ),
    dfs_csv,  # Seu DataFrame
    verbose=True,
    agent_type="openai-tools",
    max_iterations=10,  # Limita o número de operações
    allow_dangerous_code=True,  # Permite execução controlada
    handle_parsing_errors=True,  # Melhor tratamento de erros
    early_stopping_method="generate"  # Interrompe se demorar muito
)


# Exemplo de uso
resposta = agent_executor.invoke({"input": "Quantos registros existem?"})
print(resposta["output"])

pergunta = "Quantas notas fiscais existem no total?"
resposta = agent_executor.invoke({"input": pergunta})

print(f"Pergunta: {pergunta}")
print(f"Resposta: {resposta.get('output')}")



                                CHAVE DE ACESSO  \
0  41240106267630001509550010035101291224888487   
1  50240129843878000170550010000025251000181553   
2  50240129843878000170550010000025251000181553   
3  50240129843878000170550010000025251000181553   
4  50240112977901000117550010000051831659469117   

                                              MODELO  SÉRIE   NÚMERO  \
0  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1  3510129   
1  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
2  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
3  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
4  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     5183   

                   NATUREZA DA OPERAÇÃO         DATA EMISSÃO  \
0  Outras Entradas - Dev Remessa Escola  2024-01-18 07:10:39   
1    VENDA DE MERCADORIA FORA DO ESTADO  2024-01-26 11:24:42   
2    VENDA DE MERCADORIA FORA DO ESTADO  2024-01-26 11:24:42   
3   

/usr/local/lib/python3.11/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(



Invoking: `python_repl_ast` with `{'query': 'len(df1)'}`


565
Invoking: `python_repl_ast` with `{'query': 'len(df2)'}`


100Existem 565 registros no primeiro dataframe (`df1`) e 100 registros no segundo dataframe (`df2`).

> Finished chain.
Existem 565 registros no primeiro dataframe (`df1`) e 100 registros no segundo dataframe (`df2`).


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# Para o primeiro dataframe (df1)\ndf1['CHAVE DE ACESSO'].nunique()"}`
responded: Para determinar o número total de notas fiscais, podemos verificar o número de linhas únicas na coluna "CHAVE DE ACESSO" em cada dataframe, pois cada chave de acesso representa uma nota fiscal única.

Vou calcular isso para você.

100
Invoking: `python_repl_ast` with `{'query': "# Para o segundo dataframe (df2)\ndf2['CHAVE DE ACESSO'].nunique()"}`


100Existem 100 notas fiscais no primeiro dataframe (df1) e 100 notas fiscais no segundo dataframe (df2). Portanto, o total de notas fiscais

In [26]:
pergunta = "o nome do item com maior valor e me apresente esse valor ?"
resposta = agent_executor.invoke({"input": pergunta})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.loc[df['VALOR TOTAL'].idxmax(), ['DESCRIÇÃO DO PRODUTO/SERVIÇO', 'VALOR TOTAL']]"}`


NameError: name 'df' is not defined
Invoking: `python_repl_ast` with `{'query': "df1.loc[df1['VALOR TOTAL'].idxmax(), ['DESCRIÇÃO DO PRODUTO/SERVIÇO', 'VALOR TOTAL']]"}`
responded: Parece que você não definiu o nome do dataframe que contém os itens. Vou verificar os dataframes disponíveis e encontrar o item com o maior valor.

DESCRIÇÃO DO PRODUTO/SERVIÇO    HGC 4 Branco/Branco-300-CP
VALOR TOTAL                                       985050.0
Name: 564, dtype: objectO item com o maior valor é **"HGC 4 Branco/Branco-300-CP"**, com um valor total de **R$ 985.050,00**.

> Finished chain.
